In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.9 MB/s eta 0:00:00


##library

In [2]:
from google.colab import files
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import pdfplumber

In [3]:
def upload_file():
    # Unggah file
    uploaded = files.upload()
    for file_name in uploaded.keys():
        print(f"File {file_name} berhasil diunggah.")
    return uploaded

uploaded_files = upload_file()

Saving data_magang.csv to data_magang.csv
File data_magang.csv berhasil diunggah.


In [4]:
data_magang = pd.read_csv('data_magang.csv')

In [5]:
data_magang.head(10)

,no,Bidang,Skill,Company
0,1,Data Intern (Hybrid),"SQL, Python, R, Tableau, Power BI",LUXEHOUZE
1,2,Data Analyst,"Python, SQL",cmlabs
2,3,Data Product Manager Intern - Business Intelli...,"SQL, database, Google Sheets, Slides",Shopee
3,4,Data Scientist,Natural Language Processing (NLP),cmlabs
4,5,Data Scientist Intern,"PyTorch, Deep Learning, Computer Vision",GoTo Group
5,6,Supply Analyst Intern,"SQL, Python, R, Tableau, Power BI",RedDoorz
6,7,Data Engineer Intern,"SQL, Databases",PT Lion Super Indo
7,8,Data Analyst Intern,"SQL, Python, Matlab, R, SAS",JALA
8,9,Data Analyst Intern (Revenue Operations),"SQL, Python or R,DBT, Python, Github, Google L...",wati
9,10,Data Engineering Training & Internship,"Python, Java, or SQL, MySQL, PostgreSQL",Refonte Learning


##preprocessing

In [6]:
df = data_magang.drop('no', axis=1)
df.head(10)

,Bidang,Skill,Company
0,Data Intern (Hybrid),"SQL, Python, R, Tableau, Power BI",LUXEHOUZE
1,Data Analyst,"Python, SQL",cmlabs
2,Data Product Manager Intern - Business Intelli...,"SQL, database, Google Sheets, Slides",Shopee
3,Data Scientist,Natural Language Processing (NLP),cmlabs
4,Data Scientist Intern,"PyTorch, Deep Learning, Computer Vision",GoTo Group
5,Supply Analyst Intern,"SQL, Python, R, Tableau, Power BI",RedDoorz
6,Data Engineer Intern,"SQL, Databases",PT Lion Super Indo
7,Data Analyst Intern,"SQL, Python, Matlab, R, SAS",JALA
8,Data Analyst Intern (Revenue Operations),"SQL, Python or R,DBT, Python, Github, Google L...",wati
9,Data Engineering Training & Internship,"Python, Java, or SQL, MySQL, PostgreSQL",Refonte Learning


In [7]:
# Fungsi untuk membersihkan teks
def preprocess_text(text):
    """
    Membersihkan teks dengan menghapus tanda baca dan mengubah teks menjadi huruf kecil.

    Args:
        text (str): Teks yang akan diproses.

    Returns:
        str: Teks yang telah diproses (huruf kecil dan tanpa tanda baca).
    """
    text = text.lower()  # Ubah ke lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Hapus tanda baca (non-alphanumeric characters)
    return text

# Misalnya data_magang adalah DataFrame yang telah diunggah
# df = data_magang.drop('no', axis=1)  # Jika sudah menghapus kolom 'no'

# Terapkan preprocess_text ke kolom 'Bidang' dan 'Skill'
df['Bidang'] = df['Bidang'].apply(preprocess_text)
df['Skill'] = df['Skill'].apply(preprocess_text)

# Tampilkan 10 baris pertama untuk memastikan perubahan
df.head(10)


,Bidang,Skill,Company
0,data intern hybrid,sql python r tableau power bi,LUXEHOUZE
1,data analyst,python sql,cmlabs
2,data product manager intern business intellig...,sql database google sheets slides,Shopee
3,data scientist,natural language processing nlp,cmlabs
4,data scientist intern,pytorch deep learning computer vision,GoTo Group
5,supply analyst intern,sql python r tableau power bi,RedDoorz
6,data engineer intern,sql databases,PT Lion Super Indo
7,data analyst intern,sql python matlab r sas,JALA
8,data analyst intern revenue operations,sql python or rdbt python github google looker...,wati
9,data engineering training internship,python java or sql mysql postgresql,Refonte Learning


## train model CBF

In [8]:
# Fungsi untuk melatih model content-based filtering
def train_model(df):
    """
    Melatih model content-based filtering berdasarkan dataset magang.

    Args:
        df (pd.DataFrame): DataFrame yang berisi dataset magang dengan kolom yang sudah diproses.

    Returns:
        None: Model akan disimpan sebagai magang_model.pkl
    """
    # Menggabungkan kolom 'Bidang' dan 'Skill' untuk fitur
    df['combined_features'] = df['Bidang'] + ' ' + df['Skill']

    # Melatih dan fine-tuning TF-IDF Vectorizer
    # BAGIAN YANG BISA di Fine-tuning
    tfidf = TfidfVectorizer(
        stop_words='english',
        max_features=1500,   # Membatasi jumlah fitur menjadi 1000 kata
        ngram_range=(1, 2),  # Menggunakan unigram dan bigram bisa pakai (1, 2) (1, 3) (1, 5)
        # min_df=5,            # Mengabaikan kata-kata yang muncul lebih sedikit dari 5 kali
        max_df=0.9,
        sublinear_tf=True    # Menggunakan transformasi log untuk frekuensi
    )
    tfidf_matrix = tfidf.fit_transform(df['combined_features'])

    # Output untuk melihat hasil
    print("Model telah dilatih.")
    print(f"Jumlah fitur yang dipelajari oleh model: {len(tfidf.get_feature_names_out())}")
    print(f"Shape matriks TF-IDF: {tfidf_matrix.shape}")

    # Simpan model dan data penting
    with open('magang_model.pkl', 'wb') as file:
        pickle.dump({'model': tfidf, 'tfidf_matrix': tfidf_matrix, 'data': df}, file)

    print("Model berhasil disimpan.")

# Misalkan df adalah DataFrame yang sudah diproses
train_model(df)  # Melatih model berdasarkan data yang sudah diproses


Model telah dilatih.
Jumlah fitur yang dipelajari oleh model: 587
Shape matriks TF-IDF: (64, 587)
Model berhasil disimpan.


In [9]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    """
    Extracts text content from a PDF file using pdfplumber.

    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str: The extracted text content.
    """
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [10]:
def recommend(cv_path, top_n=5):
    """
    Memberikan rekomendasi magang berdasarkan CV mahasiswa.

    Args:
        cv_path (str): Path ke file PDF CV mahasiswa.
        top_n (int): Jumlah rekomendasi teratas.

    Returns:
        pd.DataFrame: DataFrame dengan rekomendasi bidang dan perusahaan, beserta kata-kata yang relevan dan skor kemiripan.
    """
    # Memuat model yang telah dilatih
    with open('magang_model.pkl', 'rb') as file:
        saved_model = pickle.load(file)

    tfidf = saved_model['model']
    tfidf_matrix = saved_model['tfidf_matrix']
    df = saved_model['data']

    # Membaca CV mahasiswa dan mengekstrak teks
    cv_content = extract_text_from_pdf(cv_path)

    # Transformasi CV mahasiswa ke dalam bentuk TF-IDF
    cv_tfidf = tfidf.transform([cv_content])

    # Menghitung kemiripan cosine antara CV dan dataset magang
    cosine_sim = cosine_similarity(cv_tfidf, tfidf_matrix)

    # Mendapatkan rekomendasi berdasarkan kemiripan tertinggi
    top_indices = cosine_sim[0].argsort()[-top_n:][::-1]  # Menyortir indeks berdasarkan kemiripan tertinggi

    # Cek jika tidak ada rekomendasi yang relevan
    if cosine_sim[0][top_indices[0]] == 0:  # Jika kemiripan tertinggi adalah 0 (tidak ada kecocokan)
        return pd.DataFrame({"Message": ["Tidak ada rekomendasi yang relevan berdasarkan CV Anda."]})

    recommendations = df.iloc[top_indices][['Bidang', 'Company', 'Skill']]

    # Menambahkan kolom untuk kata-kata relevan dari CV dan deskripsi magang
    relevant_bidang_words = []
    relevant_skill_words = []
    similarity_scores = []

    for index in top_indices:
        # Ambil deskripsi bidang dan skill dari dataset magang
        magang_bidang = df.iloc[index]['Bidang']
        magang_skill = df.iloc[index]['Skill']

        # Tokenisasi CV dan deskripsi magang menjadi kata-kata unik
        cv_words = set(cv_content.split())
        bidang_words = set(magang_bidang.split())
        skill_words = set(magang_skill.split())

        # Temukan kata-kata yang ada di CV dan deskripsi bidang
        common_bidang_words = cv_words.intersection(bidang_words)

        # Temukan kata-kata yang ada di CV dan deskripsi skill
        common_skill_words = cv_words.intersection(skill_words)

        # Ambil 5 kata yang paling relevan (dari bidang dan skill)
        relevant_bidang_words.append(" ".join(list(common_bidang_words)[:5]))  # Ambil 5 kata pertama yang cocok
        relevant_skill_words.append(" ".join(list(common_skill_words)[:5]))  # Ambil 5 kata pertama yang cocok

        # Tambahkan skor kemiripan
        similarity_scores.append(cosine_sim[0][index])

    # Menambahkan kolom kata relevan untuk bidang dan skill di luar loop
    recommendations['Relevan_Bidang_Kata'] = relevant_bidang_words
    recommendations['Relevan_Skill_Kata'] = relevant_skill_words
    recommendations['Similarity_Score'] = similarity_scores

    return recommendations

In [11]:
def evaluate_precision_recall_at_k(recommended_df, ground_truth_df, top_n):
    """
    Menghitung Precision at K dan Recall at K.
    """
    relevant_recommendations = 0
    recommended_top_n = recommended_df.head(top_n)

    # Precision at K
    for index, rec in recommended_top_n.iterrows():
        if rec['Bidang'] in ground_truth_df['Bidang'].values:
            relevant_recommendations += 1
    precision_at_k = relevant_recommendations / top_n if top_n > 0 else 0

    # Recall at K
    total_relevant = len(ground_truth_df)
    recall_at_k = relevant_recommendations / total_relevant if total_relevant > 0 else 0

    return precision_at_k, recall_at_k


def average_precision_at_k(recommended_df, ground_truth_df, top_n):
    """
    Menghitung Average Precision at K.
    """
    relevant_recommendations = 0
    ap = 0
    for k in range(1, top_n+1):
        recommended_top_k = recommended_df.head(k)
        relevant_recommendations = sum([1 if rec['Bidang'] in ground_truth_df['Bidang'].values else 0
                                        for _, rec in recommended_top_k.iterrows()])

        # Precision pada k
        precision_at_k = relevant_recommendations / k
        ap += precision_at_k

    # Menghitung rata-rata precision pada top_n
    ap_at_k = ap / top_n if top_n > 0 else 0
    return ap_at_k

def extract_ground_truth(cv_path, df):
    """
    Mengambil bidang yang relevan dari CV mahasiswa berdasarkan data magang.

    Args:
        cv_path (str): Path ke file PDF CV mahasiswa.
        df (pd.DataFrame): DataFrame magang yang berisi bidang yang tersedia.

    Returns:
        pd.DataFrame: DataFrame dengan bidang yang relevan untuk mahasiswa berdasarkan CV.
    """
    # Mengambil teks dari CV mahasiswa
    cv_content = extract_text_from_pdf(cv_path)

    # Melakukan transformasi TF-IDF terhadap CV mahasiswa
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['Bidang'] + ' ' + df['Skill'])

    cv_tfidf = tfidf.transform([cv_content])

    # Menghitung cosine similarity antara CV dan dataset magang
    cosine_sim = cosine_similarity(cv_tfidf, tfidf_matrix)

    # Mendapatkan indeks bidang yang relevan berdasarkan kemiripan tertinggi
    top_indices = cosine_sim[0].argsort()[-top_n_value:][::-1]  # Menyortir 10 bidang relevan
    ground_truth = df.iloc[top_indices][['Bidang']]  # Ambil 10 bidang teratas

    return ground_truth

##CV1.pdf - RANIA


In [12]:

# Upload file CV mahasiswa
print("Silakan upload file CV Anda (PDF):")
uploaded_cv = files.upload()  # Mengunggah file CV mahasiswa
cv_path = list(uploaded_cv.keys())[0]  # Ambil nama file CV yang diunggah

# Memberikan rekomendasi berdasarkan CV mahasiswa
top_n_value = 5
recommendations1 = recommend(cv_path, top_n=top_n_value)

# Menampilkan rekomendasi
print(f"Top 5 rekomendasi magang:")
print(recommendations1)


Silakan upload file CV Anda (PDF):


Saving Rania Aprilia Dwi Setya Putri-resume (2).pdf to Rania Aprilia Dwi Setya Putri-resume (2).pdf
Top 5 rekomendasi magang:
                                   Bidang                         Company  \
9   data engineering training  internship                Refonte Learning   
38          market research  data analyst  PT. Motor Sights International   
61                sistem analis untuk erp            Pilarmedia Indonesia   
39                          it internship            PT Grha Dana Bersama   
4                   data scientist intern                      GoTo Group   

                                                Skill Relevan_Bidang_Kata  \
9                 python java or sql mysql postgresql            training   
38  data analytics time management organizational ...            research   
61  system design accounting logistics management ...                       
39  communication skills adaptability system analy...                  it   
4               pytorch de

##evaluasi akurasi

In [13]:
# Mengambil ground truth berdasarkan CV mahasiswa
ground_truth_df1 = extract_ground_truth(cv_path, df)
# Menghitung evaluasi Precision, Recall, dan MAP
precision_at_k1, recall_at_k1 = evaluate_precision_recall_at_k(recommendations1, ground_truth_df1, top_n=top_n_value)
ap_at_k1 = average_precision_at_k(recommendations1, ground_truth_df1, top_n=top_n_value)

# Menampilkan hasil evaluasi
print(f"Precision at {top_n_value} = {precision_at_k1:.4f}")
print(f"Recall at {top_n_value} = {recall_at_k1:.4f}")
print(f"Average Precision at {top_n_value} = {ap_at_k1:.4f}")


Precision at 5 = 0.8000
Recall at 5 = 0.8000
Average Precision at 5 = 0.9600


##CV2.pdf - mona







In [14]:
# Upload file CV mahasiswa
print("Silakan upload file CV Anda (PDF):")
uploaded_cv = files.upload()  # Mengunggah file CV mahasiswa
cv_path = list(uploaded_cv.keys())[0]  # Ambil nama file CV yang diunggah

# Memberikan rekomendasi berdasarkan CV mahasiswa
top_n_value = 5
recommendations2 = recommend(cv_path, top_n=top_n_value)

# Menampilkan rekomendasi
print(f"Top 5 rekomendasi magang:")
print(recommendations2)


Silakan upload file CV Anda (PDF):


Saving CV MONALISA MAHARANI.pdf to CV MONALISA MAHARANI.pdf
Top 5 rekomendasi magang:
                                   Bidang                         Company  \
39                          it internship            PT Grha Dana Bersama   
44                 database administrator   amIT Global Solutions Sdn Bhd   
26                            data analis      PT.WIJAYA INOVASI GEMILANG   
9   data engineering training  internship                Refonte Learning   
38          market research  data analyst  PT. Motor Sights International   

                                                Skill     Relevan_Bidang_Kata  \
39  communication skills adaptability system analy...                           
44           database admin aoag microsoft sql server  database administrator   
26  big data analysis statistical data analysis da...                           
9                 python java or sql mysql postgresql             engineering   
38  data analytics time management organizatio

In [15]:
# Mengambil ground truth berdasarkan CV mahasiswa
ground_truth_df2 = extract_ground_truth(cv_path, df)
# Menghitung evaluasi Precision, Recall, dan MAP
precision_at_k2, recall_at_k2 = evaluate_precision_recall_at_k(recommendations2, ground_truth_df2, top_n=top_n_value)
ap_at_k2 = average_precision_at_k(recommendations2, ground_truth_df2, top_n=top_n_value)

# Menampilkan hasil evaluasi
print(f"Precision at {top_n_value} = {precision_at_k2:.4f}")
print(f"Recall at {top_n_value} = {recall_at_k2:.4f}")
print(f"Average Precision at {top_n_value} = {ap_at_k2:.4f}")


Precision at 5 = 0.6000
Recall at 5 = 0.6000
Average Precision at 5 = 0.7033


##CV3.pdf - sarly

In [16]:
# Upload file CV mahasiswa
print("Silakan upload file CV Anda (PDF):")
uploaded_cv = files.upload()  # Mengunggah file CV mahasiswa
cv_path = list(uploaded_cv.keys())[0]  # Ambil nama file CV yang diunggah

# Memberikan rekomendasi berdasarkan CV mahasiswa
top_n_value = 5
recommendations3 = recommend(cv_path, top_n=top_n_value)

# Menampilkan rekomendasi
print(f"Top 5 rekomendasi magang:")
print(recommendations3)


Silakan upload file CV Anda (PDF):


Saving CV_Fath' Hani Sarli Bajsair-newww.pdf to CV_Fath' Hani Sarli Bajsair-newww.pdf
Top 5 rekomendasi magang:
                                        Bidang                       Company  \
31                                 data analis  CV. AXELERASI MAXIMA DIGITAL   
22                                data analyst                          UENA   
26                                 data analis    PT.WIJAYA INOVASI GEMILANG   
34                                 data analis          PT Alba Unggul Metal   
54  data engineer data visualization developer                  CV Dea Group   

                                                Skill Relevan_Bidang_Kata  \
31  sql machine learning statistical data analysis...         data analis   
22          data visualization operations analyst sql                data   
26  big data analysis statistical data analysis da...         data analis   
34  statistical data analysis data analytics big d...         data analis   
54  sql teamwork micro

In [17]:
# Mengambil ground truth berdasarkan CV mahasiswa
ground_truth_df3 = extract_ground_truth(cv_path, df)
# Menghitung evaluasi Precision, Recall, dan MAP
precision_at_k3, recall_at_k3 = evaluate_precision_recall_at_k(recommendations3, ground_truth_df3, top_n=top_n_value)
ap_at_k3 = average_precision_at_k(recommendations3, ground_truth_df3, top_n=top_n_value)

# Menampilkan hasil evaluasi
print(f"Precision at {top_n_value} = {precision_at_k3:.4f}")
print(f"Recall at {top_n_value} = {recall_at_k3:.4f}")
print(f"Average Precision at {top_n_value} = {ap_at_k3:.4f}")


Precision at 5 = 0.8000
Recall at 5 = 0.8000
Average Precision at 5 = 0.9600


##MAP SCORE

In [18]:
ap_scores = [ap_at_k1, ap_at_k2, ap_at_k3]
# Menghitung MAP (rata-rata dari semua AP)
map_score = sum(ap_scores) / len(ap_scores) if len(ap_scores) > 0 else 0

print(f"Mean Average Precision (MAP): {map_score:.4f}")

Mean Average Precision (MAP): 0.8744
